In [1]:
# import constants and helpers
%run Constants.ipynb

In [6]:
from sidecar import Sidecar
import ipywidgets as widgets
from ipyleaflet import (
    Map, ImageOverlay, DrawControl, WidgetControl
)
from matplotlib import pyplot as plt

def get_draw_control():
    draw_control = DrawControl()
    draw_control.polyline =  {}
    draw_control.polygon = {
        # "shapeOptions": {
        #     "fillColor": "#6be5c3",
        #     "color": "#6be5c3",
        #     "fillOpacity": 1.0
        # },
        "allowIntersection": False
    }
    draw_control.circle = {}
    draw_control.rectangle = {}
    draw_control.circlemarker = {}
    return draw_control


def create_button_widgets(*, options: dict, position='bottomright'):
    buttons = []
    for name, content in options.items():
        button = widgets.Button(
            description=name,
            disabled=False,
            button_style=content.get('style', 'info'), # 'success', 'info', 'warning', 'danger' or ''
            tooltip=name,
            layout=widgets.Layout(width='100%', height='100%'),
            # icon='check' # (FontAwesome names without the `fa-` prefix)
        )
        button.on_click(content.get('callback'))
        buttons.append(button)
    hbox_widget = widgets.HBox(buttons)
    widget_control = WidgetControl(widget=hbox_widget, position=position)
    return widget_control


def create_map(*args, **kwargs) -> Map:
    default = dict(
        zoom=19,
        # zoom_control=True,
        # min_zoom=18,
        max_zoom=25,
        # dragging=False,
    )
    default.update(kwargs)
    m = Map(**default)
    sidecar = Sidecar(title='Map')
    with sidecar:
        display(m)
    draw_control = get_draw_control()
    m.add_control(draw_control)
    
    return m, sidecar, draw_control

In [10]:
import rasterio as rio
from rasterio.plot import reshape_as_raster, reshape_as_image

def load_image(img_src, *, bands=[3,1,2], input_crs=CRS_2056, output_crs=CRS_4326, reduction_factor=6000):
    with rio.open(img_src, 'r') as src:
        bounds = shapely_to_geo_series_crs_conversion(shape=box(*src.bounds), input_crs=input_crs, output_crs=output_crs)
        center = [bounds.centroid.y,bounds.centroid.x]
        image_data = src.read([3,1,2])
    reduced = reduce_depth(image_data, display_min=0, display_max=reduction_factor)
    return reduced, bounds, center
    
def show_next_image_on_map(image, *, m: Map, overlay, draw):
    data = draw_control.get_state().get('data')
    
    if overlay:
        m.remove_layer(overlay)
    image_reduced, bounds, center = load_image(image)
    bounds = bounds.bounds
    image_bounds = [[bounds[3], bounds[2]],[bounds[1], bounds[0]]]
    map_image = reshape_as_image(image_reduced)
    im_overlay = make_imageoverlay(map_image, image_bounds)
    m.add_layer(im_overlay)
    m.zoom = 20
    m.center = center
    # m.fit_bounds(image_bounds)
    return im_overlay

In [11]:
RAPPPI_CENTER = [47.2245967,8.8168816]
m, sidecar, draw_control = create_map(center=RAPPPI_CENTER)

# ugly state
im_overlay = None
crossing = IMAGES_TIF_FOLDER_CROSSINGS.glob("*.tif")

In [12]:
im_overlay = show_next_image_on_map(next(crossing), m=m, overlay=im_overlay, draw=draw_control)

def print_callback(*args, **kwargs):
    global im_overlay
    im_overlay = show_next_image_on_map(next(crossing), m=m, overlay=im_overlay, draw=draw_control)
    print(args, kwargs)

buttons = {
    'next': {
        "callback": print_callback,
        "style": "info",
    },
    'prev': {
        "callback": print_callback,
        "style": "info",
    }
}
button_control = create_button_widgets(options=buttons)
m.add_control(button_control)

In [8]:
im_overlay = show_next_image_on_map(next(crossing), m=m, overlay=im_overlay, draw=draw_control)

[]


In [ ]:
draw_control.get_state().get('data')

In [ ]:
# todo: get data from draw_control
draw_control.get_state().get('data')
m.remove_layer(im_overlay)

In [ ]:
sidecar.close()

# Appendix

## find a decent reduction value


In [ ]:
reduction_factor = 6000

EXAMPLE_IMAGE = IMAGES_TIF_FOLDER_CROSSINGS / "8.474699974060059_47.40547180175781_is_crossing_1.tif"
EXAMPLE_IMAGE.exists()

import geopandas as gpd
import rasterio as rio
from shapely.geometry import box
from rasterio.plot import reshape_as_raster, reshape_as_image



with rio.open(EXAMPLE_IMAGE, 'r') as src:
    bounds = shapely_to_geo_series_crs_conversion(shape=box(*src.bounds), input_crs=CRS_2056, output_crs=CRS_4326)
    wgs84_center = [bounds.centroid.y,bounds.centroid.x]
    image_data = src.read([3,1,2])
    reduced = reduce_depth(image_data, display_min=0, display_max=reduction_factor)
rio.plot.show(reduced)

In [ ]:
image_bounds = bounds.bounds
image_bounds = [[image_bounds[3], image_bounds[2]],[image_bounds[1], image_bounds[0]]]
print(wgs84_center)
print(image_bounds)